### Monitoramento de KPIs de Desempenho e Insights para táxis amarelos em Nova York

Conexão com banco Postgres Docker

In [70]:
import sys
import os
import psycopg2
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Adicionar a raiz do projeto ao sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../NYC_Yellow_Taxi_Trip_Records")))

import config.database_config as db_config

def connect_postgres():
# Conexão banco de dados
    connection = psycopg2.connect(
                host=db_config.DB_HOST,
                database=db_config.DB_NAME,
                user=db_config.DB_USER,
                password=db_config.DB_PASSWORD,
                port=db_config.DB_PORT
            )

    cursor = connection.cursor()
    cursor.execute(f'SET search_path TO {db_config.DB_SCHEMA}')
    return connection
# connection.autocommit = True
connection = connect_postgres()


#### KPIs

1. Receita média por corrida e por dia

In [71]:
connection = connect_postgres()

query = """
WITH daily_revenue AS (
    SELECT
        DATE(td."date") AS trip_date,
        COUNT(trip_id) AS total_trips,
        SUM(total_amount) AS daily_total_revenue,
        SUM(total_amount) / COUNT(trip_id) AS avg_revenue_per_trip 
    FROM trips_fact tf
    LEFT JOIN time_dim td 
    ON tf.pickup_time_id = td.time_id 
    GROUP BY (trip_date)
),
avg_revenue_last_7_days AS (
    SELECT
        trip_date,
        daily_total_revenue,
        total_trips,
        avg_revenue_per_trip,
        CAST(
            AVG(daily_total_revenue) OVER (
                ORDER BY trip_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
            ) AS NUMERIC(10,2)
        ) AS avg_revenue_last_7_days
    FROM daily_revenue
)
SELECT 
    trip_date,
    CAST(daily_total_revenue AS NUMERIC(10,2)),
    total_trips,
    CAST(avg_revenue_per_trip AS NUMERIC(6,2)),
    avg_revenue_last_7_days,
    RANK() OVER (ORDER BY avg_revenue_per_trip DESC) AS avg_revenue_per_trip_rank,
    RANK() OVER (ORDER BY avg_revenue_last_7_days DESC) AS avg_revenue_last_7_days_rank
FROM avg_revenue_last_7_days
ORDER BY trip_date;
"""

pd.read_sql_query(query, connection)


,trip_date,daily_total_revenue,total_trips,avg_revenue_per_trip,avg_revenue_last_7_days,avg_revenue_per_trip_rank,avg_revenue_last_7_days_rank
0,2023-12-31,183.48,7,26.21,183.48,27,32
1,2024-01-01,2155472.85,68541,31.45,1077828.16,1,31
2,2024-01-02,2228037.51,71559,31.14,1461231.28,2,30
3,2024-01-03,2308344.18,78518,29.40,1673009.50,3,29
4,2024-01-04,2731564.38,97865,27.91,1884720.48,13,28
5,2024-01-05,2648587.88,97617,27.13,2012031.71,21,27
6,2024-01-06,2322135.26,89755,25.87,2056332.22,30,26
7,2024-01-07,1845525.41,63333,29.14,2319952.50,4,24
8,2024-01-08,2162875.25,75999,28.46,2321009.98,7,23
9,2024-01-09,2196374.29,84642,25.95,2316486.66,28,25


2. Distância média percorrida por corrida

In [72]:
connection = connect_postgres()

query = """
SELECT 
    CAST(AVG(trip_distance) AS NUMERIC(5,2)) AS avg_distance_per_trip
FROM trips_fact;
"""

pd.read_sql_query(query, connection)

,avg_distance_per_trip
0,3.26


3. Taxa de ocupação média por corrida

In [73]:
connection = connect_postgres()

query = """
SELECT 
    CAST(AVG(passenger_count) AS NUMERIC(5,2))AS avg_passenger_count
FROM trips_fact;
"""

pd.read_sql_query(query, connection)

,avg_passenger_count
0,1.35


4. Tempo médio das corridas por dia

In [74]:
connection = connect_postgres()

query = """
WITH trip_durations AS (
    SELECT
        DATE(td.date) AS trip_date,
        (calc_trip_duration_seconds/60) AS trip_duration
	FROM trips_fact tf
	LEFT JOIN time_dim td 
	on tf.pickup_time_id = td.time_id     
)
SELECT
    trip_date,
    CAST(AVG(trip_duration) AS NUMERIC(4,2)) AS avg_trip_duration_minutes
FROM trip_durations
GROUP BY trip_date
ORDER BY avg_trip_duration_minutes DESC;
"""

pd.read_sql_query(query, connection)

,trip_date,avg_trip_duration_minutes
0,2024-01-02,16.46
1,2024-01-25,16.27
2,2024-01-24,16.17
3,2024-01-16,16.16
4,2024-01-01,16.14
5,2024-01-03,16.08
6,2024-01-12,16.00
7,2024-01-11,15.92
8,2024-01-17,15.87
9,2024-01-26,15.80


5. Proporção dos métodos de pagamento utilizados

In [75]:
connection = connect_postgres()

query = """
SELECT 
    ptd.payment_description,
    COUNT(tf.trip_id) AS total_trips,
    ROUND(COUNT(tf.trip_id) * 100.0 / SUM(COUNT(tf.trip_id)) OVER (), 2) AS percentage_usage
FROM trips_fact tf
JOIN payment_type_dim ptd ON tf.payment_type_id = ptd.payment_type_id
GROUP BY ptd.payment_description
ORDER BY total_trips DESC;
"""

pd.read_sql_query(query, connection)


,payment_description,total_trips,percentage_usage
0,Credit Card,2293321,83.21
1,Cash,425327,15.43
2,Dispute,24830,0.90
3,No Charge,12747,0.46


#### Insights

6. Períodos de maior e menor demanda (corridas por hora do dia)

In [76]:
connection = connect_postgres()

query = """
WITH hourly_demand AS (
    SELECT
        td.hour AS hour_of_day,
        COUNT(tf.trip_id) AS total_trips
	FROM trips_fact tf
	LEFT JOIN time_dim td 
	ON tf.pickup_time_id = td.time_id
    GROUP BY td.hour

)
SELECT
    hour_of_day,
    total_trips,
    RANK() OVER (ORDER BY total_trips DESC) AS demand_rank
FROM hourly_demand
ORDER BY demand_rank;
"""

pd.read_sql_query(query, connection)

,hour_of_day,total_trips,demand_rank
0,18,197947,1
1,17,193370,2
2,16,180609,3
3,15,179158,4
4,14,173324,5
5,19,172309,6
6,13,160958,7
7,12,155412,8
8,20,150803,9
9,21,149934,10


7. Locais de embarque e desembarque mais populares

In [77]:
connection = connect_postgres()

query = """
SELECT 
    l1.borough AS pickup_location,
    l2.borough AS dropoff_location,
    COUNT(t.trip_id) AS total_trips
FROM trips_fact t
LEFT JOIN location_dim l1 ON t.pickup_location_id = l1.location_id
LEFT JOIN location_dim l2 ON t.dropoff_location_id = l2.location_id
GROUP BY pickup_location, dropoff_location
ORDER BY total_trips DESC
LIMIT 10;
"""

pd.read_sql_query(query, connection)

,pickup_location,dropoff_location,total_trips
0,Manhattan,Manhattan,2319903
1,Queens,Manhattan,145845
2,Manhattan,Queens,69752
3,Queens,Queens,58918
4,Manhattan,Brooklyn,49309
5,Queens,Brooklyn,39378
6,Brooklyn,Brooklyn,9928
7,Manhattan,Bronx,7662
8,Queens,Outside of NYC,7556
9,Manhattan,Unknown,7023


8. Faturamento ao longo dos dias da semana (sazonalidade)

In [78]:
connection = connect_postgres()

query = """
WITH monthly_revenue AS (
    SELECT
        td.day_of_week AS day_of_week,
        td.part_of_day AS part_of_day,
        SUM(total_amount) AS total_revenue
    FROM trips_fact tf
    LEFT JOIN time_dim td 
	on tf.pickup_time_id = td.time_id   
    GROUP BY day_of_week, part_of_day
)
SELECT
    day_of_week,
    part_of_day,
    TO_CHAR(total_revenue, 'FM$999,999,999.00') AS total_revenue
FROM monthly_revenue
ORDER BY total_revenue DESC;
"""

pd.read_sql_query(query, connection)

,day_of_week,part_of_day,total_revenue
0,Wednesday,Afternoon,"$4,937,912.71"
1,Tuesday,Afternoon,"$4,685,489.51"
2,Thursday,Afternoon,"$4,223,734.98"
3,Monday,Afternoon,"$4,188,784.82"
4,Friday,Afternoon,"$3,903,424.03"
5,Saturday,Afternoon,"$3,649,580.12"
6,Sunday,Afternoon,"$3,345,922.92"
7,Wednesday,Evening,"$3,281,231.76"
8,Wednesday,Morning,"$3,127,025.29"
9,Tuesday,Morning,"$3,100,285.77"


9. Relação entre tipo de tarifa e faturamento médio

In [79]:
connection = connect_postgres()

query = """
SELECT
    rcd.rate_code_description,
    COUNT(tf.trip_id) AS total_trips,
    CAST(AVG(tf.total_amount) AS NUMERIC(6, 2)) AS avg_revenue
FROM trips_fact tf
LEFT JOIN rate_code_dim rcd
ON tf.rate_code_id = rcd.rate_code_id
GROUP BY rcd.rate_code_description
ORDER BY avg_revenue DESC;
"""

pd.read_sql_query(query, connection)

,rate_code_description,total_trips,avg_revenue
0,Nassau or Westchester,6088,131.12
1,Newark,7519,115.72
2,JFK,95797,93.05
3,Negotiated fare,18239,85.78
4,Standard rate,2628577,24.26
5,Group ride,5,6.60
